In [ ]:
import re
import pandas as pd
import os
from openai import OpenAI

# OpenAI API 키 설정
api_key = "your-key"
client = OpenAI(api_key=api_key)

# 사용자 이름을 자동으로 가져오기
username = os.getlogin()

# 바탕화면 경로 설정
desktop_path = os.path.join(os.path.expanduser('~'), 'Desktop')

# 대봉공급처 폴더 경로 설정
supplier_folder = os.path.join(desktop_path, '대봉공급처', '도매A-026')

# 엑셀 파일 이름
file_name = '0604_대봉_견적서.xlsx'
file_path = os.path.join(supplier_folder, file_name)

def read_excel(file_path):
    """엑셀 파일을 읽고 DataFrame을 반환"""
    df = pd.read_excel(file_path)
    return df.to_csv(index=False)

def call_gpt_function(data):
    """OpenAI ChatCompletion API를 호출하여 데이터를 변환"""
    messages = [
        {"role": "system", "content": """
            You are an expert in transforming Excel data into a 2D list format.
            Please provide the data as a 2D list in Python list format based on the given structure.
            Columns to consider:
            - A: Option Name
            - B: KG
            - C: Size Info and Number of Fruits
            - E: Price Including Delivery
            - G: Weight Per Size
            Example output:
            [
                ['Supplier Code', 'Option Name', 'KG', 'Price Including Delivery', 'Size/Number of Fruits/Weight Per Size'],
                ['WholesaleA-026', 'Option A', '1kg', '6000', 'Small/15 units/Each 65g']
            ]
        """},
        {"role": "user", "content": data}
    ]
    functions = [
        {
            "name": "transform_excel_data",
            "description": "Transforms Excel CSV data to a 2D list.",
            "parameters": {
                "type": "object",
                "properties": {
                    "data": {
                        "type": "string",
                        "description": "The CSV data from the Excel file"
                    }
                },
                "required": ["data"]
            }
        }
    ]
    
    response = client.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        functions=functions,
        function_call="auto"
    )
    
    return response

def clean_gpt_output(output):
    # GPT 출력에서 불필요한 부분을 제거하고 올바른 리스트 형식으로 변환
    cleaned_output = output.strip()
    # 리스트 형식을 올바르게 닫아줌
    if cleaned_output.count('[') > cleaned_output.count(']'):
        cleaned_output += ']'
    return cleaned_output

try:
    # 엑셀 파일 열기
    excel_file = pd.ExcelFile(file_path)
    
    # 모든 시트의 이름 확인
    sheet_names = excel_file.sheet_names
    print("시트 이름:", sheet_names)

    # 각 시트를 읽어 데이터프레임으로 변환
    dataframes = {}
    output_data = []

    for sheet_name in sheet_names:
        excel_content = ""
        df = pd.read_excel(excel_file, sheet_name=sheet_name)
        dataframes[sheet_name] = df
        excel_content += f"Sheet: {sheet_name}\n"
        excel_content += df.to_string(index=False)
        excel_content += "\n\n"
        
        gpt_output = gpt_transformTest(excel_content)
        cleaned_output = clean_gpt_output(gpt_output)
        output_data.append(cleaned_output)
        print("GPT Output:", cleaned_output)  # GPT 원시 출력 확인

    # GPT 출력 데이터를 2차원 리스트 형식으로 변환
    data = []
    for output in output_data:
        print("Raw GPT output before parsing:", output)  # 파싱 전 출력 확인
        data.append(output)  # GPT 출력값을 그대로 2차원 리스트에 저장

    # 변환된 데이터 출력
    for item in data:
        print(item)

except FileNotFoundError:
    print("파일을 찾을 수 없습니다. 경로를 확인하세요.")
except pd.errors.EmptyDataError:
    print("파일에 데이터가 없습니다.")
except Exception as e:
    print(f"오류가 발생했습니다: {e}")

# 데이터를 2차원 리스트로 변환하는 함수
def parse_gpt_output(data):
    parsed_data = []
    for item in data:
        # 문자열에서 리스트 형식을 추출
        match = re.search(r"\[.*?\]", item)
        if match:
            # 문자열을 실제 리스트로 변환
            try:
                parsed_list = eval(match.group())
                parsed_data.append(parsed_list)
            except SyntaxError as e:
                print(f"SyntaxError while parsing: {e}")
                print(f"Problematic output: {match.group()}")
    return parsed_data

# 데이터 파싱
parsed_data = parse_gpt_output(data)

# 파싱된 데이터 출력
for row in parsed_data:
    print(row)


In [1]:
# print(type(data))
# print(type(data[1]))
# print(type(parse_gpt_output(data[1])))
# print(parse_gpt_output(data))
# print(data[1])
# print(data)

new_list = []
for value in data:
    print(value)
    new_value = parse_gpt_output(value)
    print(new_value)
    new_list.append(new_value)

print(new_list)
for i in new_list:
    print(type(i))






NameError: name 'data' is not defined

In [36]:
import os
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import os.path
import json
import pickle
import requests

def oauth():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    creds_filename = 'test-daebong-service-account.json'  # 서비스 계정 파일 경로를 지정합니다.

    # 서비스 계정 파일을 사용하여 인증 정보를 로드합니다.
    creds = service_account.Credentials.from_service_account_file(creds_filename, scopes=SCOPES)
    return creds

def oauthByWeb():
    # 필요한 스코프를 지정합니다.
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/script.projects',
        "https://www.googleapis.com/auth/script.scriptapp",
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
    ]
    creds = None
    # 'token.json' 파일이 존재하면, 저장된 인증 정보를 불러옵니다.
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = pickle.load(token)

    # 저장된 인증 정보가 없거나, 유효하지 않은 경우 새로운 인증을 진행합니다.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            creds_filename = 'oauth-app-script.json'
            flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
            creds = flow.run_local_server(port=8080)
        # 새로운 인증 정보를 'token.json'에 저장합니다.
        with open('token.json', 'wb') as token:
            pickle.dump(creds, token)

    return creds

def append_crawling_data(service, spreadsheet_id, values, sheet_name):
    # 데이터를 추가할 범위 설정
    range_name = f'{sheet_name}!E2:k100'

    body = {
        'values': values
    }

    # 스프레드시트에 데이터 추가
    result = service.spreadsheets().values().append(
        spreadsheetId=spreadsheet_id, range=range_name,
        valueInputOption='RAW', body=body, insertDataOption='INSERT_ROWS').execute()

    print(result)

def update_spreadsheet(array_2d):
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    product_sheet_id = "1cKe3qTX_C7qYqSDJvR3ab1LJFAVOMOfkDXFuL3OKNPg"
    sheet_name = "test기존업체신규품목"
    append_crawling_data(service_sheets, product_sheet_id, array_2d, sheet_name)
    print("완료")
# if __name__ == "__main__":
#     creds = oauthByWeb()
#     service_sheets = build('sheets', 'v4', credentials=creds)
#     product_sheet_id = "1cKe3qTX_C7qYqSDJvR3ab1LJFAVOMOfkDXFuL3OKNPg"
#     sheet_name = "기존업체신규품목"
#     append_crawling_data(service_sheets, product_sheet_id, data, sheet_name)

#      # 데이터를 100행씩 분할하여 전송
#     chunk_size = 10
#     for i in range(0, len(data), chunk_size):
#         chunk = data[i:i + chunk_size]
#         append_crawling_data(service_sheets, product_sheet_id, chunk, sheet_name)

In [37]:
def run_conversation(excel_data):
    # Step 1: send the conversation and available functions to the model
    messages = [ {"role": "system", "content": """
            You are an expert in transforming Excel data into a 2D list format.
            Please provide the data as a 2D list in Python list format based on the given structure.
            Columns to consider:
            - A: Option Name
            - B: KG
            - C: Size Info and Number of Fruits
            - E: Price Including Delivery
            - G: Weight Per Size
            Example output:
            [
                ['Supplier Code', 'Option Name', 'KG', 'Price Including Delivery', 'Size/Number of Fruits/Weight Per Size'],
                ['WholesaleA-026', 'Option A', '1kg', '6000', 'Small/15 units/Each 65g']
            ]
        """},
        {"role": "user", "content": excel_data}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "update_spreadsheet",
                "description": "엑셀에서 정확하게 다음과 같은 형식으로 데이터를 뽑아내는 거야.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "array_2d": {
                            "type": "array",
                            "items": {
                                "type": "array",
                                "items": {
                                    "type": "string",
                                    "description": "fruite option"
                                }
                            },
                            "description": "A 2D array of fruite option"
                        },
                    },
                    "required": ["array_2d"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        tools=tools
    )
    response_message = response.choices[0].message
     # Accessing tool_call
    tool_call = response.choices[0].message.tool_calls[0]
    print("response.choices[0]")
    print(response.choices[0])
    print("tool_call")
    print(tool_call)
    
    if tool_call:
        # Only process the first tool call
        function_name = tool_call.function.name
        print("function_name")
        print(function_name)
        function_args = json.loads(tool_call.function.arguments)
        print("function_args")
        print(function_args)
        print(function_args.get("array_2d"))
        update_spreadsheet(function_args.get("array_2d"))
        function_response = update_spreadsheet(
            array_2d=function_args.get("array_2d"),
        )

        print("response_message")
        # print(response_message)
        messages.append(response_message)  # Extend conversation with assistant's reply
        messages.append(
            {
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )  # Extend conversation with function response
        # second_response = client.chat.completions.create(
        #     model="gpt-3.5-turbo",
        #     messages=messages,
        # )  # Get a new response from the model where it can see the function response
        return "complete"

In [42]:
import re
import pandas as pd
import os
from openai import OpenAI

# OpenAI API 키 설정
api_key = "your-key"
client = OpenAI(api_key=api_key)

# 주어진 경로
base_path = "/Users/baegmingi/daebong/daebong-automation/product-upload"

# # 사용자 이름을 자동으로 가져오기
# username = os.getlogin()

# # 대봉공급처 폴더 경로 설정
# supplier_folder = os.path.join(base_path, ".")

# 엑셀 파일 이름
file_name = '0604_대봉_견적서.xlsx'
file_path = os.path.join(base_path, file_name)
def read_excel(file_path):
    """엑셀 파일을 읽고 DataFrame을 반환"""
    df = pd.read_excel(file_path)
    return df.to_csv(index=False)

try:
    # 엑셀 파일 열기
    excel_file = pd.ExcelFile(file_path)
    
    # 모든 시트의 이름 확인
    sheet_names = excel_file.sheet_names
    print("시트 이름:", sheet_names)

    # 각 시트를 읽어 데이터프레임으로 변환
    dataframes = {}
    output_data = []

    for sheet_name in sheet_names:
        print(sheet_name)
        excel_content = ""
        df = pd.read_excel(excel_file, sheet_name=sheet_name)
        dataframes[sheet_name] = df
        excel_content += f"Sheet: {sheet_name}\n"
        excel_content += df.to_string(index=False)
        excel_content += "\n\n"
        # print(excel_content)
        gpt_output = run_conversation(excel_content)

        # cleaned_output = clean_gpt_output(gpt_output)
        # output_data.append(cleaned_output)
        # print("GPT Output:", cleaned_output)  # GPT 원시 출력 확인

    # GPT 출력 데이터를 2차원 리스트 형식으로 변환
    # data = []
    # for output in output_data:
    #     print("Raw GPT output before parsing:", output)  # 파싱 전 출력 확인
    #     data.append(output)  # GPT 출력값을 그대로 2차원 리스트에 저장

    # # 변환된 데이터 출력
    # for item in data:
    #     print(item)

except FileNotFoundError as e:
    print(f"파일을 찾을 수 없습니다. 경로를 확인하세요.: {e}")
except pd.errors.EmptyDataError:
    print("파일에 데이터가 없습니다.")
except Exception as e:
    print(f"오류가 발생했습니다: {e}")

시트 이름: ['견적서', '신비 복숭아', '신선 복숭아', '천도 복숭아', '자두', '대극천(반납작 복숭아)', '하우스 수박', '하우스 복숭아', '참외', '백자메론', '가정용 사과', '완숙토마토', '샐러드채소', '청양고추', '일반 파프리카', '오이고추', '백오이', '가시오이', '당근', '양파', '감자', '비트', '다진마늘', '대추']
견적서
response.choices[0]
Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_DiG66R9keF70c7cpbWJyTBT4', function=Function(arguments='{"array_2d":[["Supplier Code","Option Name","KG","Price Including Delivery","Size/Number of Fruits/Weight Per Size"],["WholesaleA-026","Option A","1kg","6000","Small/15 units/Each 65g"]]}', name='update_spreadsheet'), type='function')]))
tool_call
ChatCompletionMessageToolCall(id='call_DiG66R9keF70c7cpbWJyTBT4', function=Function(arguments='{"array_2d":[["Supplier Code","Option Name","KG","Price Including Delivery","Size/Number of Fruits/Weight Per Size"],["WholesaleA-026","Option A","1kg","6000","Small/15 units/